# Show SED of Standards ordered by colors

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/10/31
- last update : 2025/10/31 : 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd
import os

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Wavelength sampling of.*")

In [ ]:
from getCalspec import getCalspec

In [ ]:
#!pip install adjustText

In [ ]:
import sys
sys.path.append('../lib')
#import libAtmosphericFit

In [ ]:
# This package encapsulate the calculation on calibration used in this nb
from libPhotometricCorrections import *

In [ ]:
plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
# where are stored the figures
pathfigs = "figs_TOOL03"
prefix = "tool03"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
def ShowLSSTBands(ax,alpha=0.5,am=1.2,pwv=4.,oz=300.,tau=0.,beta=1.2):
    """
    """
    pc = PhotometricCorrections(am,pwv,oz,tau,beta)

    for index,f in enumerate(filter_tagnames):
        ax.plot(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=alpha) 
        ax.fill_between(pc.bandpass_total_std[f].wavelen,pc.bandpass_total_std[f].sb,color=filter_color[index],alpha=alpha) 
        #ax.axvline(FILTERWL[index,2],color=filter_color[index],linestyle="-.")


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(6,4))

ShowLSSTBands(ax,alpha=0.1)

ax.set_xlabel("$\\lambda$ (nm)")
ax.set_title("Total filter throughput (LSST)")

## Input target file

In [ ]:
targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"
df = pd.read_csv(targets_mag_files,index_col=0)      

In [ ]:
df

### Suppress same rows

In [ ]:
# On crée une copie pour ne pas altérer df directement
df_clean = df.copy()

# Étape 1 — normaliser les noms en supprimant les espaces superflus
df_clean["spectr_name_clean"] = df_clean.index.to_series().str.replace(" ", "", regex=False)

# Étape 2 — normaliser aussi RA et Dec (en supprimant les espaces ou zéros superflus)
df_clean["RA_clean"] = df_clean["RA"].astype(str).str.replace(" ", "", regex=False)
df_clean["Dec_clean"] = df_clean["Decl"].astype(str).str.replace(" ", "", regex=False)

# Étape 3 — supprimer les doublons basés sur (spectr_name_clean, RA_clean, Dec_clean)
df_clean = df_clean.reset_index(drop=False)  # on garde l'ancien index (nom original)
df_clean = df_clean.drop_duplicates(subset=["spectr_name_clean", "RA_clean", "Dec_clean"], keep="first")

# Étape 4 — remettre l’index propre
df_clean = df_clean.set_index("index")

# Optionnel : supprimer les colonnes temporaires
df_clean = df_clean.drop(columns=["spectr_name_clean", "RA_clean", "Dec_clean"])

df_clean

In [ ]:
df =df_clean

### Sort by colors

In [ ]:
df = df.sort_values(by="B_V")
df = df.reset_index().rename(columns={'index': 'spectr_name'})

In [ ]:
NSED = len(df)

In [ ]:
#list(df.columns)

In [ ]:
df["Sp_T"].values

In [ ]:
df

## Plot color-color diagram

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from adjustText import adjust_text
import numpy as np

# Liste des types spectraux
SpT = df["Sp_T"].values

# Liste ordonnée des types spectraux uniques
unique_types = list(dict.fromkeys(SpT))
N_types = len(unique_types)

# Dictionnaire de correspondance type → indice couleur
type_to_color_idx = {t: i for i, t in enumerate(unique_types)}

# Pour chaque étoile : récupère la couleur correspondant à son type spectral
color_indices = np.array([type_to_color_idx[t] for t in SpT])

# --- Création de la colormap ---
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# --- Plot principal ---
fig, ax = plt.subplots(figsize=(6,6))
sc = ax.scatter(
    df["z-y"], df["g-r"],
    c=color_indices, cmap=cmap, norm=norm,
    s=200, edgecolor='k', label="CALSPEC"
)

ax.set_title("Color-Color diagram of CALSPEC targets")
ax.set_xlabel("Z - Y (mag)")
ax.set_ylabel("G - R (mag)")
ax.grid(True)

# --- Colorbar ---
cbar = plt.colorbar(sc, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O (bleu) en haut, M (rouge) en bas

# --- Ajout des noms avec adjustText ---
texts = []
for i, name in enumerate(df.index):
    color = cmap(norm(color_indices[i]))  # même couleur que le point
    x = df["z-y"].iloc[i]
    y = df["g-r"].iloc[i]
    
    # Décalage initial (en coordonnées du plot)
    dx, dy = 0.02, 0.02  
    texts.append(
        ax.text(
            x + dx,
            y + dy,
            df["spectr_name"].iloc[i],
            fontsize=9,
            color=color,
            weight='bold',
            va='center'
        )
    )

# Ajustement automatique pour éviter les chevauchements
adjust_text(
    texts,
    ax=ax,
    arrowprops=dict(arrowstyle="-", color='gray', lw=0.4),
    force_text=(0.6, 0.8),
    expand_points=(1.4, 1.4),
    expand_text=(1.2, 1.4),
    lim=150
)

ax.grid(which='major', linestyle='--', linewidth=0.6, alpha=0.4)
ax.grid(which='minor', linestyle=':', linewidth=0.3, alpha=0.2)
ax.minorticks_on()

figname =f"{pathfigs}/{prefix}_targetcolorcolordiagram"+figtype
plt.savefig(figname)

plt.show()


## Plot magnitude color diagram (HR)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patheffects as pe
from adjustText import adjust_text
import numpy as np

# --- Préparation des couleurs par type spectral ---
SpT = df["Sp_T"].values
unique_types = list(dict.fromkeys(SpT))  # préserve l'ordre d'apparition
N_types = len(unique_types)

# correspondance type spectral → indice couleur
type_to_color_idx = {t: i for i, t in enumerate(unique_types)}
color_indices = np.array([type_to_color_idx[t] for t in SpT])

# colormap cohérente avec le diagramme couleur-couleur
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# --- Création du plot ---
fig, ax = plt.subplots(figsize=(6, 6))

# Nuage de points coloré selon le type spectral
sc = ax.scatter(
    df["B_V"],
    df["V"],
    c=color_indices,
    cmap=cmap,
    norm=norm,
    s=200,
    marker="o",
    edgecolor="k",
    label="CALSPEC"
)

# --- Colorbar cohérente ---
cbar = plt.colorbar(sc, ax=ax, ticks=range(N_types))
cbar.ax.set_yticklabels(unique_types)
cbar.set_label("Spectral Type", fontsize=12)
cbar.ax.invert_yaxis()  # O bleu en haut, M rouge en bas

# --- Titre et axes ---
ax.set_title("Magnitude–Color Diagram of CALSPEC targets", fontsize=14)
ax.set_xlabel("B − V (mag)")
ax.set_ylabel("V (mag)")
ax.grid(True)
ax.invert_yaxis()  # plus lumineux vers le haut

# --- Ajout des labels colorés avec contour blanc ---
texts = []
for i, row in df.iterrows():
    x, y = row["B_V"], row["V"]
    name = row["spectr_name"]
    color = cmap(norm(type_to_color_idx[row["Sp_T"]]))
    
    txt = ax.text(
        x + 0.02, y + 0.05,  # léger décalage
        name,
        color=color,
        fontsize=9,
        va="center",
        weight="bold",
        path_effects=[pe.withStroke(linewidth=2.5, foreground="white")]
    )
    texts.append(txt)

# --- Ajustement automatique pour éviter les chevauchements ---
adjust_text(
    texts,
    ax=ax,
    arrowprops=dict(arrowstyle="-", color="gray", lw=0.5),
    expand_points=(1.3, 1.4),
    expand_text=(1.2, 1.3),
    force_text=(0.4, 0.4),
    lim=150
)

ax.grid(which='major', linestyle='--', linewidth=0.6, alpha=0.4)
ax.grid(which='minor', linestyle=':', linewidth=0.3, alpha=0.2)
ax.minorticks_on()


figname =f"{pathfigs}/{prefix}_targetmagnituecolordiagram"+figtype
plt.savefig(figname)

plt.show()


## Show SED grid

In [ ]:
n_stars = len(df)
ncols = 4
nrows = int(np.ceil(n_stars / ncols))
wlmin=300.
wlmax=1200    

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*2.5), sharex=True)
axes = axes.flatten()  # simplifie l’itération


for i, (idx, row) in enumerate(df.iterrows()):
    ax = axes[i]
    
    # Extraire info
    sedname = row["spectr_name"]
    SpT_type = row["Sp_T"]
    color = cmap(norm(type_to_color_idx[SpT_type]))
    
    # Charger la SED avec getCalspec
    c = getCalspec.Calspec(sedname)
    sed = c.get_spectrum_numpy()
    wave, Flambda = sed['WAVELENGTH'].value, sed['FLUX'].value
    wave = wave/10.
    Flambda = Flambda*10.   

    waveindexes = np.where(np.logical_and(wave>wlmin,wave<wlmax))[0]
    wave = wave[waveindexes]
    Flambda = Flambda[waveindexes]
    
    
    # Tracer SED
    label = f"{sedname} ({SpT_type})"
    ax.plot(wave, wave*Flambda, color=color,label=label)  # lambda*Flambda pour signal photonique
    #ax.set_title(f"{name} ({SpT_type})", fontsize=8, color=color)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.legend(loc="upper right")
    ax2 =ax.twinx()
    ShowLSSTBands(ax2,alpha=0.05)

# Supprimer les axes vides
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Axes communs
fig.text(0.5, 0.04, "Wavelength [nm]", ha="center",fontsize=12)
fig.text(0.04, 0.5, r"$\lambda F_\lambda$ [erg/cm2]", va="center", rotation="vertical",fontsize=12)

#plt.suptitle("SED $\lambda F_\lambda$ ",fontsize=16,y=1)
plt.tight_layout(rect=[0.05,0.05,1,0.95])

figname =f"{pathfigs}/{prefix}_allsedlambdaflambda"+figtype
plt.savefig(figname)

plt.show()



In [ ]:
n_stars = len(df)
ncols = 4
nrows = int(np.ceil(n_stars / ncols))
wlmin=300.
wlmax=1200    

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*3, nrows*2.5), sharex=True)
axes = axes.flatten()  # simplifie l’itération


for i, (idx, row) in enumerate(df.iterrows()):
    ax = axes[i]
    
    # Extraire info
    sedname = row["spectr_name"]
    SpT_type = row["Sp_T"]
    color = cmap(norm(type_to_color_idx[SpT_type]))
    
    # Charger la SED avec getCalspec
    c = getCalspec.Calspec(sedname)
    sed = c.get_spectrum_numpy()
    wave, Flambda = sed['WAVELENGTH'].value, sed['FLUX'].value
    wave = wave/10.
    Flambda = Flambda*10.   

    waveindexes = np.where(np.logical_and(wave>wlmin,wave<wlmax))[0]
    wave = wave[waveindexes]
    Flambda = Flambda[waveindexes]
    
    
    # Tracer SED
    label = f"{sedname} ({SpT_type})"
    ax.plot(wave, Flambda, color=color,label=label)  # lambda*Flambda pour signal photonique
    #ax.set_title(f"{name} ({SpT_type})", fontsize=8, color=color)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    ax.legend(loc="upper right")
    ax2 =ax.twinx()
    ShowLSSTBands(ax2,alpha=0.05)

# Supprimer les axes vides
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Axes communs
fig.text(0.5, 0.04, "Wavelength [nm]", ha="center")
fig.text(0.04, 0.5, r"$F_\lambda$ [erg/cm2/nm]", va="center", rotation="vertical")

plt.tight_layout(rect=[0.05,0.05,1,0.95])

figname =f"{pathfigs}/{prefix}_allsedflambda"+figtype
plt.savefig(figname)

plt.show()

